<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/extract/gradioprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install easyocr gradio pythainlp langchain langchain_huggingface langchain_community pytesseract transformers
!sudo apt-get install ghostscript

Processing triggers for man-db (2.10.2-1) ...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

resume_directory = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
pdf_files = [os.path.join(resume_directory, f) for f in os.listdir(resume_directory) if f.endswith('.pdf')]

# Print the list of PDF files to verify
print(pdf_files)

['/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume baa.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 6.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 7.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 1.pdf']


## **Job Qualification**

| **ᴊᴏʙ** | **ᴘᴏꜱɪᴛɪᴏɴ** | **ᴄᴏᴜɴᴛ** |
|:---:|:---:|:---:|
| 1 | ʙᴜꜱɪɴᴇꜱꜱ ᴀɴᴀʟʏꜱᴛ | 5 |
| 2 | ᴀɪ ᴇɴɢɪɴᴇᴇʀ | 7 |


In [4]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient
import pytesseract
from nltk.tokenize import sent_tokenize

In [42]:
resume_prompt = """
# Resume Evaluation Prompt

**You are a career coach and resume evaluator**, tasked with thoughtfully assessing a candidate's resume based on a specific job description. Your goal is to strike a balance between the candidate’s qualifications and their potential for growth. Keep in mind that real-world hiring involves not only meeting immediate requirements but also identifying long-term potential and adaptability.

## Instructions:
- **Focus primarily on the job description**, but also consider any unique skills and experience that the candidate offers.
- Your evaluation should prioritize candidates who align well with the role but remain open to those who might provide unexpected strengths.
- **Don’t rely solely on a perfect match**—instead, take a holistic view of the candidate to assess their overall fit, potential, and growth trajectory.

### Evaluation Criteria:

1. **Strong Fit (PASS)**:
   - The candidate aligns well with the job requirements and demonstrates strong potential to excel, even if some qualifications are missing.
   - Consider their **unique strengths**, **relevant experience**, and **transferable skills**.
   - If they show adaptability and a strong overall fit, classify them as "Strong Fit."

2. **Promising (RECONSIDER)**:
   - The candidate aligns with some aspects of the job but has notable gaps.
   - However, their experience suggests they could grow into the role.
   - Consider them as a candidate with potential, particularly if their overall background suggests they would be worth interviewing or reviewing further.

3. **Not a Fit (UNRELATED)**:
   - The candidate’s background does not align with the job description and there is no significant potential for the role.
   - Their experience, while potentially strong in other areas, is unrelated to the requirements for this position.

## Evaluation Process:

1. **Step 1**: Carefully review the **job description** and extract the core requirements and preferred qualifications.
2. **Step 2**: Compare the **resume** to the job description:
   - Identify relevant qualifications and skills.
   - Note any **transferable skills** or **unique strengths** that the candidate brings.
3. **Step 3**: Consider the candidate's **overall potential**, adaptability, and long-term growth.
4. **Step 4**: Classify the resume and provide feedback.

## Output Instructions:
- Summarize the candidate's resume in two key sections: **ข้อดี (Strengths)** and **ข้อควรปรับปรุง (Areas for Improvement)**.
- Then provide the **Classification** (Pass / Reconsider / Unrelated) and a **Concise Rationale**.
- Do **not** include a detailed analysis of the job description or resume in the output.

### What Not to Do:
- **Don’t disqualify candidates** for not meeting every single requirement. Assess their ability to adapt and grow.
- **Don’t ignore transferable skills** that could be useful in the role.
- **Remain open-minded** to candidates who may bring unique value, even if their experience isn’t a perfect match.

## Selection Criteria:

- **Relevance to Job Requirements**: Focus on how well the candidate’s experience and skills align with both the required and preferred qualifications of the job description.
- **Transferable Skills**: Identify skills and experiences that, while not directly related, could be beneficial in the role.
- **Potential**: Look beyond immediate requirements to assess whether the candidate shows promise to grow into the role and contribute long-term.
- **Holistic View**: Evaluate the resume with a broader perspective, considering adaptability, cultural fit, and overall value to the team or company.

---

### **Few-Shot Examples** (for internal reference, never copy them)

#### **Job Description**:
- Required: 5+ years of experience in software development, proficiency in Python and JavaScript, experience with cloud platforms, and leadership skills. Bachelor's degree in Computer Science or related field.
- Preferred: Master's degree, DevOps experience, and familiarity with modern software development practices.

#### **Resume**:
- Candidate has 6 years of software development experience, proficient in Python and JavaScript, and cloud platform experience. Holds a Master’s degree in Computer Science and has led teams of up to 10 people. Some exposure to DevOps.

#### **Evaluation**:

**Classification**: Pass
**Rationale**: The candidate meets the core requirements and brings valuable experience in leadership and DevOps. They have demonstrated the ability to succeed in a complex environment, making them a strong fit for the role.

**ข้อดี (Strengths)**:
- Extensive software development experience and proficiency in required programming languages.
- Leadership experience and familiarity with cloud platforms.

**ข้อควรปรับปรุง (Areas for Improvement)**:
- Limited exposure to DevOps, which is a preferred qualification.

---

#### **Job Description**:
- Required: 3+ years of software development experience, proficiency in Python and JavaScript, experience with cloud platforms. Bachelor’s degree required.
- Preferred: Experience with DevOps and React.js.

#### **Resume**:
- The candidate has 3 years of experience in automation and Python development. They have worked on cloud-based projects but have less direct experience with JavaScript or React.js.

#### **Evaluation**:

**Classification**: Reconsider
**Rationale**: The candidate shows strong Python and cloud platform experience but lacks direct JavaScript experience. Given their expertise in automation and cloud projects, they might still be a good fit with some support in JavaScript.

**ข้อดี (Strengths)**:
- Strong experience with automation and Python.
- Familiarity with cloud platforms.

**ข้อควรปรับปรุง (Areas for Improvement)**:
- Lacks direct experience with JavaScript and React.js, which are mentioned in the job description.
---

## **Job Description**:
{jobdescribe}

## **Resume**:
{resume}

## **Template for Evaluation**:

**Classification**: (Pass / Reconsider / Unrelated)
**Rationale**: Provide a clear and concise explanation for your classification. Focus on how well the candidate aligns with the job description, highlight any transferable skills or potential, and explain any significant gaps. Mention both strengths and areas for improvement.

**ข้อดี (Strengths)**:
- Summarize the key strengths of the candidate based on their resume.

**ข้อควรปรับปรุง (Areas for Improvement)**:
- Summarize areas where the candidate could improve or is lacking in relation to the job description.
"""


In [43]:
# Define the model parameters
model_params = {
    "max_new_tokens": 500,
    "temperature": 0.1,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}
semaphore = Semaphore(50)

def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",
            f"-r{dpi}",
            "-o", output_format,
            pdf_file
        ]
        subprocess.run(gs_command, check=True)
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files
    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

def chunk_text(text, tokenizer, max_tokens=200):
    tokens = tokenizer.tokenize(text)
    total_tokens = len(tokens)
    chunks = []
    start = 0
    while start < total_tokens:
        end = min(start + max_tokens, total_tokens)
        chunk = tokenizer.convert_tokens_to_string(tokens[start:end])
        chunks.append(chunk)

        start += max_tokens

    return chunks

def tag_and_clean_text(text, tagger, tokenizer, unwanted_pattern, max_tokens=200):
    text_chunks = chunk_text(text, tokenizer, max_tokens=max_tokens)
    tagged_text = []
    cleaned_text = []

    for i, chunk in enumerate(text_chunks):
        ner = tagger.get_ner(chunk, tag=True)
        if not ner:
            ner = chunk

        # Clean the tags
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]*?</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner, flags=re.IGNORECASE)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        # Append the tagged and cleaned text
        tagged_text.append(ner)
        cleaned_text.append(cleaned_ner)
    # Combine results from all chunks
    combined_tagged_text = "\n".join(tagged_text).strip()
    combined_cleaned_text = "\n".join(cleaned_text).strip()

    return combined_tagged_text, combined_cleaned_text


# Function to generate an answer using the prompt
def generate_answer(resume, job_description):
    formatted_prompt = resume_prompt.replace("{resume}", resume).replace("{jobdescribe}", job_description)
    truncated_prompt = formatted_prompt[:model_params["max_new_tokens"]]
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

# Extract classification from the result text
def extract_classification(text):
    pattern = r'\*\*\s*Classification\s*\**:\s*(?:\[)?\s*(Pass|Reconsider|Not Pass|Unrelated)\s*(?:\])?'
    match = re.search(pattern, text, re.IGNORECASE)
    if not match:
        pattern = r'Classification\s*:\s*(?:\[)?\s*(Pass|Reconsider|Not Pass|Unrelated)\s*(?:\])?'
        match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1)
    return text

def process_file(file, job_description, is_pdf=True):
    if is_pdf:
        images = convert_pdf_to_images(file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"
    else:
        raw_text = extract_text_from_image(file.name)

    tagger = NamedEntityTagger()
    tokenizer = tagger.tokenizer

    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me', 'birth', 'date',
        'address', 'email.', 'ประวัติ'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    tagged_text, cleaned_text = tag_and_clean_text(raw_text, tagger, tokenizer, unwanted_pattern)
    evaluation = generate_answer(cleaned_text, job_description)
    result = extract_classification(evaluation)

    df = pd.DataFrame([{
        "File": os.path.basename(file.name),
        "Raw_Text": raw_text,
        "Tagged_Text": tagged_text,
        "Cleaned_Text": cleaned_text,
        "Job_Description": job_description,
        "Evaluation": evaluation,
        "Result": result
    }])
    return df

# Function to process multiple files (remains unchanged)
def process_multiple_files(files, job_description, is_pdf=True):
    all_results = pd.DataFrame()
    for file in files:
        df = process_file(file, job_description, is_pdf)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Modify the function to append results without loading the existing DataFrame
def save_dataframe(df, save_path='/content/output.csv'):
    try:
        # Append new data directly to the CSV without reloading existing data
        df.to_csv(save_path, mode='a', header=not os.path.exists(save_path), index=False, encoding='utf-8-sig')
        return save_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Function to process a batch of DataFrame rows
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    job_description = row.get('Job_Description', "")
    result = generate_answer(resume, job_description)
    return result

# Function to process a batch of rows
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

In [58]:
# Modify the Gradio interface
def gradio_interface(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty):
    try:
        # Update model parameters
        model_params.update({
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty
        })
        df = process_multiple_files(files, job_description, is_pdf)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Result'] = df['Evaluation'].apply(extract_classification)
        csv_path = save_dataframe(df)
        df_output = df[["File", "Result", "Evaluation"]]  # Keep the Evaluation column for internal use
        return df_output, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

import gradio as gr

def create_gradio_interface():
    # Define the custom theme
    custom_theme = gr.themes.Default(
        primary_hue=gr.themes.colors.red,
        secondary_hue=gr.themes.colors.pink
    )

    with gr.Blocks(theme=custom_theme) as demo:
        with gr.Row():
            # Left Column for Input Section
            with gr.Column(scale=1):
                file_input = gr.Files(label="Upload Files (PDFs or images)", file_count="multiple")
                job_input = gr.Textbox(placeholder="Enter the job description here...", label="Job Description", lines=10)
                submit_btn = gr.Button("Process")

                # Model parameters
                with gr.Column(visible=True):
                    is_pdf_input = gr.Checkbox(label="PDF", value=True)
                    max_new_tokens_input = gr.Slider(label="Max New Tokens", minimum=50, maximum=1024, value=250, step=50)
                    temperature_input = gr.Slider(label="Temperature", minimum=0.01, maximum=1.0, value=0.1, step=0.1)
                    top_p_input = gr.Slider(label="Top P", minimum=0.1, maximum=1.0, value=0.95, step=0.05)
                    repetition_penalty_input = gr.Slider(label="Repetition Penalty", minimum=0.1, maximum=2.0, value=1.0, step=0.1)

            # Right Column for Output Section
            with gr.Column(scale=1):
                # Add instruction text above the DataFrame
                instruction_text = gr.Markdown("**Click on the candidate's resume file to see the evaluation detail**")

                # Output DataFrame with limited rows visible and interactivity on
                output_df = gr.DataFrame(
                    headers=["File", "Result"],
                    type="pandas",
                    interactive=False
                )

                # Create a text component to display the evaluate data of the selected row
                eval_text = gr.Textbox(label="Evaluation detail", interactive=False)

                # Download button for CSV
                output_csv = gr.DownloadButton(label="Download CSV")

        # Function to process inputs and generate output
        def on_submit(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty):
            global df  # Make df accessible globally for the select event
            df, csv_path = gradio_interface(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty)
            return df[["File", "Result"]], csv_path

        # Function to display the evaluation of a selected row
        def show_evaluation(evt: gr.SelectData):
            row_index = evt.index[0]  # Get the index of the selected row
            selected_eval = df.iloc[row_index]["Evaluation"]  # Extract evaluation for the selected row
            return str(selected_eval)  # Ensure this is converted to string

        # Function to clear the evaluation detail
        def clear_evaluation():
            return ""

        # Bind the submit button to the processing function
        submit_btn.click(on_submit,
                         inputs=[file_input, job_input, is_pdf_input, max_new_tokens_input, temperature_input, top_p_input, repetition_penalty_input],
                         outputs=[output_df, output_csv])

        # Bind the row click event to show evaluation data
        output_df.select(show_evaluation, outputs=eval_text)
    return demo

In [ ]:
app = create_gradio_interface()
app.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fbcbb658e2c2580ac0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
## software enginner (https://th.jobsdb.com/job/78728644?type=standout&ref=search-standalone#sol=56ea90da2ce55925ca5b5294f7dc6009051ee403)
## business analysis (https://th.jobsdb.com/Business-Analysis-jobs?jobId=78724436&type=standout)
## AI Engineer (https://th.jobsdb.com/AI-Engineer-jobs?jobId=78686025&type=standout)

In [ ]:
import pandas as pd
a = pd.read_csv('/content/output.csv')
a

,File,Raw_Text,Tagged_Text,Cleaned_Text,Job_Description,Evaluation,Result
0,RESUME AI ENG.pdf,"narakorn vorakul\n18/4 pak nam, mmueang samut ...",<PERSON>nar</PERSON><PERSON>akorn vorakul</PER...,email. ai engineer ตําแหน่ง summary enthusia...,Job Title: Machine Learning / Artificial Intel...,**Feedback for Candidate**: Offer constructive...,Reconsider
1,Resume_Chanatip_Buthsri.pdf,"1t. samed., sukhumvit rd.\n19/3 moo\na. muang ...",<LOCATION>1</LOCATION><LOCATION>t</LOCATION><L...,mobile a recent data science graduate with div...,Key Responsibilities:\n\nDevelop and Deploy ML...,**Strengths**: List the candidate's key streng...,Unrelated
2,Resume.pdf,kantaphong ma veekul\ntell: 093-451-6206\nemai...,<PERSON>kan</PERSON><PERSON>tap</PERSON><PERSO...,. line im hard working and responsible to work...,Key Responsibilities:\n\nDevelop and Deploy ML...,**Strengths**: List the candidate's key streng...,Unrelated
3,Resume.pdf,kantaphong ma veekul\ntell: 093-451-6206\nemai...,<PERSON>kan</PERSON><PERSON>tap</PERSON><PERSO...,. line im hard working and responsible to work...,Responsibilities\nFormulate real-world problem...,**Feedback for Candidate**: Offer constructive...,Reconsider
4,Resume_Chanatip_Buthsri.pdf,"1t. samed., sukhumvit rd.\n19/3 moo\na. muang ...",<LOCATION>1</LOCATION><LOCATION>t</LOCATION><L...,mobile a recent data science graduate with div...,Job Responsibilities\n\nดำเนินการตามแผนงานและค...,**Feedback**: Offer constructive feedback on t...,**Feedback**: Offer constructive feedback on t...
5,Resume_Chanatip_Buthsri.pdf,"1t. samed., sukhumvit rd.\n19/3 moo\na. muang ...",<LOCATION>1</LOCATION><LOCATION>t</LOCATION><L...,mobile a recent data science graduate with div...,Data Engineer / Data Analytics (Junior/Senior)...,**Feedback**: Offer constructive feedback on t...,Reconsider


In [ ]:
a.to_excel('/content/drive/MyDrive/AIEngineer/resume/JOB/output.xlsx')

In [ ]:
a['Result'].value_counts()

,count
Result,
Unrelated,33
Reconsider,17
Pass,10


## **Pass**
จะตัดสินใจให้ `ผ่าน` ก็ต่อเมื่อผู้สมัครดูมี potential จาก resume ดูจะ applied ไปกับ job description ได้หรือตรงตาม requirement เป็นส่วนใหญ่

In [ ]:
a[a['Result']=='Pass']

,File,Raw_Text,Tagged_Text,Cleaned_Text,Job_Description,Evaluation,Result
0,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,good,**Feedback for Candidate**: Offer constructive...,Pass
1,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Pass
12,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,**Machine Learning / Artificial Intelligence E...,**Feedback for Candidate**: Offer constructive...,Pass
23,Resume ba3.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทํางาน มุ่งมันที่จะใช้ความรู้และท...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Pass
31,Resume ba3.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทํางาน มุ่งมันที่จะใช้ความรู้และท...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Pass
32,Resume ba4.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม business analyst เป้าหมายในการทํางาน ต้อ...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Pass
33,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Pass
38,Resume ba1.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการ...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Pass
40,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Pass
50,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,**Machine Learning / Artificial Intelligence E...,**Feedback for Candidate**: Offer constructive...,Pass


## **Reconsider**
จะตัดสินใจให้ `พิจารณา` ก็ต่อเมื่อผู้สมัคร จาก resume ดูจะมีแนวโน้มที่สามารถ applied ตาม requirement ได้ ควรสัมภาษณ์อีกครั้ง

In [ ]:
a[a['Result']=='Reconsider']

,File,Raw_Text,Tagged_Text,Cleaned_Text,Job_Description,Evaluation,Result
4,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Reconsider
6,resume ai8.pdf,alexander jones\nartificial intelligence engin...,<PERSON>alexander</PERSON> jones artificial in...,jones artificial inigence engineer innovative ...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Reconsider
9,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,**Machine Learning / Artificial Intelligence E...,**Feedback for Candidate**: Offer constructive...,Reconsider
13,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,**Machine Learning / Artificial Intelligence E...,**Feedback for Candidate**: Offer constructive...,Reconsider
14,resume ai8.pdf,alexander jones\nartificial intelligence engin...,<PERSON>alexander</PERSON> jones artificial in...,jones artificial inigence engineer innovative ...,**Machine Learning / Artificial Intelligence E...,**Feedback for Candidate**: Offer constructive...,Reconsider
17,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Reconsider
24,Resume ba4.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม business analyst เป้าหมายในการทํางาน ต้อ...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Reconsider
25,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Reconsider
39,Resume ba5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ มุ่งมันในการนําทักษะด้านการจัดกา...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Reconsider
43,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Reconsider


## **Unrelated**
จะตัดสินใจให้ `ไม่เกี่ยวข้อง` ก็ต่อเมื่อผู้สมัคร จาก resume ดูจะ ไม่เกี่ยวข้องกับ job description หรือตรงข้ามกับ requirement เป็นส่วนใหญ่

In [ ]:
a[a['Result']=='Unrelated']

,File,Raw_Text,Tagged_Text,Cleaned_Text,Job_Description,Evaluation,Result
2,resume hr2.pdf,โคร์นา\nบีจานูเอวา\n เก็บวก์บฉ็น\nความสดใสร่า...,<PERSON>โ</PERSON><PERSON>โค</PERSON><PERSON>ร...,เก็บวก์บฉ็น ความสดใสร่าเริงคือตัวตนของฉัน ฉันช...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Unrelated
3,resume hr3.pdf,แอนนา แคทรีนา มาร์เคซี\nพนักงานฝ่ายบุคคล\n อาย...,<PERSON>แ</PERSON><PERSON>แอนนา แคทรีนา มาร์เค...,พนักงานฝ่ายบุคคล ข้อมูลโดยย่อ การศึกษา ระดับ ...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Unrelated
5,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback**: Offer constructive feedback on t...,Unrelated
7,Resume ba3.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทํางาน มุ่งมันที่จะใช้ความรู้และท...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Unrelated
8,Resume ba4.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม business analyst เป้าหมายในการทํางาน ต้อ...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback for Candidate**: Offer constructive...,Unrelated
10,resume hr2.pdf,โคร์นา\nบีจานูเอวา\n เก็บวก์บฉ็น\nความสดใสร่า...,<PERSON>โ</PERSON><PERSON>โค</PERSON><PERSON>ร...,เก็บวก์บฉ็น ความสดใสร่าเริงคือตัวตนของฉัน ฉันช...,**Machine Learning / Artificial Intelligence E...,**Feedback for Candidate**: Offer constructive...,Unrelated
11,resume hr3.pdf,แอนนา แคทรีนา มาร์เคซี\nพนักงานฝ่ายบุคคล\n อาย...,<PERSON>แ</PERSON><PERSON>แอนนา แคทรีนา มาร์เค...,พนักงานฝ่ายบุคคล ข้อมูลโดยย่อ การศึกษา ระดับ ...,**Machine Learning / Artificial Intelligence E...,**Feedback for Candidate**: Offer constructive...,Unrelated
16,Resume ba4.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม business analyst เป้าหมายในการทํางาน ต้อ...,**Machine Learning / Artificial Intelligence E...,**Feedback for Candidate**: Offer constructive...,Unrelated
18,resume hr2.pdf,โคร์นา\nบีจานูเอวา\n เก็บวก์บฉ็น\nความสดใสร่า...,<PERSON>โ</PERSON><PERSON>โค</PERSON><PERSON>ร...,เก็บวก์บฉ็น ความสดใสร่าเริงคือตัวตนของฉัน ฉันช...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Unrelated
21,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,**Business Analyst**\n\nWe are seeking a new g...,**Feedback for Candidate**: Offer constructive...,Unrelated
